## <center> Lab: UDS over CAN
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" />
### <center> Instructor: Dr. Jeremy Daily

# Prerequisites
We'll need a file that logs traffic from a unified diagnostic service (UDS) session. UDS is used by many companies, and is especially popular among european companies. For our example, we'll use a DDEC Reports session.

DDEC Reports is a freely available fleet management tool for Detroit Diesel Engines. If you wanted to use it, you'd need an RP1210 compliant vehicle diagnostics adapter (VDA), like the DPA5. A google search for "DDEC Reports Download" usually points you in the right direction. 


We've already gathered a DDEC Reports file and the candump log file for when it was made. We need to download that first. If you are running this notebook on a BeagleBone, make sure it is connected to the Internet. 

In [1]:
# Install the requests library so we can download files from the Internet
# uncomment the following line if requests is not installed
#%pip install requests --upgrade --user
#%pip install --upgrade pip
import requests
import struct

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Download the candump log file from Dr. Daily's website
url = r'https://www.engr.colostate.edu/~jdaily/J1939/files/candump_DDECReportsExtaction032819.txt'
r = requests.get(url)
candump_file = r.text
print(candump_file[:203])
# Write the received text data to a file:
with open('DDECReportsExtaction032819.txt','w') as log_file:
    log_file.write(candump_file)

(1531228460.025007) can1 18FDD001#FFFFFF3030FFFFFF
(1531228460.025565) can1 14FEF131#CFFFFFF300FFFF30
(1531228460.026155) can1 18FEDB01#E900E900E900FFFF
(1531228460.026721) can1 18FD9401#4C01FFFFFFFFFFFF


In [3]:
# Download the DDEC Reports file from Dr. Daily's website
url = r'https://www.engr.colostate.edu/~jdaily/J1939/files/DDECReportsExtaction032819.pdf'
r = requests.get(url)
# Write the received binary pdf data to a file:
with open('DDECReportsExtaction032819.pdf','wb') as pdf_file:
    pdf_file.write(r.content)

If the block above successfully ran, then you should be able to open the PDF file with this link [DDECReportsExtaction032819.pdf](DDECReportsExtaction032819.pdf)

In [4]:
# Download the DDEC Reports file from Dr. Daily's website
url = r'https://www.engr.colostate.edu/~jdaily/J1939/files/DDECReportsExtaction032819.XTR'
r = requests.get(url)
# Write the received binary DDEC Reports data to a file:
with open('DDECReportsExtaction032819.XTR','wb') as xtr_file:
    xtr_file.write(r.content)

# ISO 14229 or ISO 15765
These ISO standards define a method of communication that is primarily focused on diagnosic services. 

For SAE J1939 networks, there is a special parameter group number 55808 (0xDA00) that is used for all UDS traffic. This means we can filter out all the other traffic and only focus on the UDS messages.

Source address 241 (0xF1) is the world-wide harmonized (WWH) off-board diagnositics (OBD) Tester for OBD tester as defined in WWH-OBD ISO 15765-4.


In [5]:
# Extract UDS messages from the CAN log
uds_messages = []
for line in candump_file.split('\n'):
    elements = line.replace("#"," ").split()
    try:
        if "18DA" in elements[2]:
            can_id = int(elements[2],16) 
            sa = can_id & 0xFF
            da = (can_id & 0xFF00) >> 8
            data = bytes.fromhex(elements[3])
            timestamp = float(elements[0].strip('()'))
            uds_messages.append({'timestamp':timestamp,'sa':sa,'da':da,'data':data})
    except IndexError:
        # This is usually an empty string at the end
        print(elements)
uds_messages[:5]

[]


[{'timestamp': 1531228470.611006,
  'sa': 241,
  'da': 0,
  'data': b'\x03"\xf1\x00\xff\xff\xff\xff'},
 {'timestamp': 1531228470.631073,
  'sa': 0,
  'da': 241,
  'data': b'\x07b\xf1\x00\x02\x01\x0e\x01'},
 {'timestamp': 1531228472.603126,
  'sa': 241,
  'da': 0,
  'data': b'\x02\x10\x03\x00\x00\x00\x00\x00'},
 {'timestamp': 1531228472.621486,
  'sa': 0,
  'da': 241,
  'data': b'\x06P\x03\x00\x14\x00\xc8\x01'},
 {'timestamp': 1531228473.115446,
  'sa': 241,
  'da': 0,
  'data': b'\x03"\xf1\x00\x00\x00\x00\x00'}]

With only the ISO 15765 data in the list, we can begin to understand some of the diagnostic communications. Let's take this one line at a time. For reference, we can use the ISO specification (which is expensive) or this poster:

https://automotive.softing.com/fileadmin/sof-files/pdf/de/ae/poster/UDS_Faltposter_softing2016.pdf

Since all of the UDS information is in the data field, we only derive source and destination address from the CAN ID. The first nibble of the first byte in the data frame is one of 3 things:
1. If the first nibble is a 0, then it is a single frame message. The second nibble is the message size.
2. If the first nibble is a 1, then this is the first frame of a long message. The next 12 bits represent the message size.
3. If the first nibble is a 2, then this is a consecutive frame from the first message.
4. If the first nibble is a 3, then it is a session control message. 

https://en.wikipedia.org/wiki/ISO_15765-2

Since 12 bits are allocated to describe the message size, the total amount of data that can be transfered in 1 session is 4096 bytes, which is over twice the size of the maximum for the J1939 Transport Protocol.

### Single Frame Message: Read Data By Identifier

In [6]:
# Let's look at the first UDS message:
uds_messages[0]

{'timestamp': 1531228470.611006,
 'sa': 241,
 'da': 0,
 'data': b'\x03"\xf1\x00\xff\xff\xff\xff'}

This message comes from source address 241, which is the OBD Tool.

In [7]:
# We can pretty-print the data:
" ".join(["{:02X}".format(b) for b in uds_messages[0]['data']])

'03 22 F1 00 FF FF FF FF'

The first nibble is a zero, so this is a single frame message. 

The next nibble is 3, which means there are 3 bytes to this message, which are 0x22, 0xF1, 0x00

The first byte in this sequence is called the *service identifier* (SID). The upper right table in the poster shows the different service ids.

![UDS_Service_Overview.png](UDS_Service_Overview.png)

Based on this message the SID is 0x22, which is the **Read Data By Identifier** service name (RDBI). It does not have a sub function, so the identifier to read is 0xF100.

When we look up the 0xF100 identifier in Appendix C.1 of the ISO 14229-3 standard, it says that this parameter is Manufacturer Specific, which means we may not get any other information about the data that comes next. 

In [8]:
# Look at the next record
print(uds_messages[1])

{'timestamp': 1531228470.631073, 'sa': 0, 'da': 241, 'data': b'\x07b\xf1\x00\x02\x01\x0e\x01'}


In [9]:
# This is the response to
" ".join(["{:02X}".format(b) for b in uds_messages[1]['data']])

'07 62 F1 00 02 01 0E 01'

This response from the engine controller (SA = 0) is a single frame with seven bytes: 0x07.

The response service identifier is indicated by adding 0x40 to the requested SID. This is the same as setting the the 6th bit: 0b01000000


In [10]:
sid = 0x22
sid_response = sid | 0x40 #set the repsonse bit
print("response SID: 0x{:02X}".format(sid_response))

response SID: 0x62


The parameter for the response is still 0xF100 and the data for the response is 0x02010E01. We don't know what this parameter from the engine means, but presumably the diagnostic software application knows what to do.  

In [11]:
# let's look at the next sequence of messages
for msg in uds_messages[2:10]:
    print(msg)

{'timestamp': 1531228472.603126, 'sa': 241, 'da': 0, 'data': b'\x02\x10\x03\x00\x00\x00\x00\x00'}
{'timestamp': 1531228472.621486, 'sa': 0, 'da': 241, 'data': b'\x06P\x03\x00\x14\x00\xc8\x01'}
{'timestamp': 1531228473.115446, 'sa': 241, 'da': 0, 'data': b'\x03"\xf1\x00\x00\x00\x00\x00'}
{'timestamp': 1531228473.131437, 'sa': 0, 'da': 241, 'data': b'\x07b\xf1\x00\x02\x01\x0e\x03'}
{'timestamp': 1531228473.828286, 'sa': 241, 'da': 0, 'data': b"\x02'\x05\x00\x00\x00\x00\x00"}
{'timestamp': 1531228473.841246, 'sa': 0, 'da': 241, 'data': b'\x04g\x05\x11\x8e\x01\x0e\x03'}
{'timestamp': 1531228474.095685, 'sa': 241, 'da': 0, 'data': b"\x04'\x06\xc0,\x00\x00\x00"}
{'timestamp': 1531228474.111126, 'sa': 0, 'da': 241, 'data': b'\x02g\x06\x11\x8e\x01\x0e\x03'}


### Session Control Message
The next message is a single frame, 2-byte message from the OBD Tester device (the diagnostic tool).

In [12]:
print(uds_messages[2])

{'timestamp': 1531228472.603126, 'sa': 241, 'da': 0, 'data': b'\x02\x10\x03\x00\x00\x00\x00\x00'}


In [13]:
" ".join(["{:02X}".format(b) for b in uds_messages[2]['data']])

'02 10 03 00 00 00 00 00'

This message SID is 0x10, which is the Diagnostic Session Control (DSC). It has a subfunction, which is indicated by the following byte, 0x03. The highest two bits of the value are both zero. According to the Sub-Function Byte of UDS, this means the suppressPosRspMsgIndicatioBit is not set, so the ECU is expected to respond.

When looking up SID of 0x10, we find the subfunction of this SID is the diagnostic Session Type. A value of 0x30 means 
```
extendedDiagnosticSession (EXTDS)
```
This diagnosticSession can be used to enable all diagnostic services required to support the adjustment of functions like "Idle Speed, CO Value, etc." in the server's memory. It can also be used to enable diagnostic services, which are not specifically tied to the adjustment of functions according to the ISO-14229-3 standard.

So we are looking at an extended diagnostic session.


In [14]:
# Look at the response:
print(uds_messages[3])
" ".join(["{:02X}".format(b) for b in uds_messages[3]['data']])

{'timestamp': 1531228472.621486, 'sa': 0, 'da': 241, 'data': b'\x06P\x03\x00\x14\x00\xc8\x01'}


'06 50 03 00 14 00 C8 01'

The response from the engine controller to the OBD tool is a single frame with 6 bytes (0x06). The response SID is has 0x40 added to the origination SID: 0x50 = 0x10 + 0x40.

The diagnostic Function Type is still 0x03, which is an extended session. This takes up 2 bytes, there are still four to go.

According to ISO 14229, Tables 26-28, we can interpet the next four bytes as the sessionParameterRecord, which contain ECU timing requirements. These bytes have the following definitions:
```
P2Server_max (high byte)
P2Server_max (low byte)
P2*Server_max (high byte)
P2*Server_max (low byte)
```
These parameters have the following definitions:

| Parameter | Description | # of bytes | Resolution | minimum value | maximum value |
| :-- | :-- | --- | --- | --- | --- |
| `P2Server_max` |Default P2Server_max timing supported by the server for the activated diagnostic session. | 2 | 1 ms | 0 ms | 65,535 ms |
| `P2*Server_max` | Enhanced (NRC 0x78) P2Server_max supported by the server for the activated diagnostic session. | 2 | 10 ms | 0 ms | 655,350 ms |

Enhanced Negative Response Code (NRC) 0x78 is `requestCorrectlyReceived-ResponsePending` (RCRRP)

```This NRC indicates that the request message was received correctly, and that all parameters in the request message were valid, but the action to be performed is not yet completed and the server is not yet ready to receive another request. As soon as the requested service has been completed, the server shall send a positive response message or negative response message with a response code different from this.```

In [15]:
P2Server_max = struct.unpack('>H',uds_messages[3]['data'][3:5])[0]
print("The ECU can support up to {} milliseconds for P2Server_max".format(P2Server_max))

The ECU can support up to 20 milliseconds for P2Server_max


In [16]:
P2_star_Server_max = struct.unpack('>H',uds_messages[3]['data'][5:7])[0]
print("The ECU can support up to {} milliseconds for P2*Server_max".format(P2_star_Server_max*10))

The ECU can support up to 2000 milliseconds for P2*Server_max


## Security Session Setup

In [17]:
# look at the transactions
for i in range(6,10):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  02 27 05 00 00 00 00 00
From:   0, To: 241,  04 67 05 11 8E 01 0E 03
From: 241, To:   0,  04 27 06 C0 2C 00 00 00
From:   0, To: 241,  02 67 06 11 8E 01 0E 03


The first message is from the OBD Tester to the engine controller of a single frame, 2 byte message. The SID is 0x27, which is **Security Access**. The Sub-Function is 0x05, which is **requestSeed**. The tool is asking the ECU for a seed so it can authenticate itself to perform diagnostics. 

The second message is the response from the engine with only 4 bytes. The Response code is 0x67 = 0x27 + 0x40 and the positive response message is the **securitySeed**. This is a 16-bit number with the high byte first and low byte second (big endian, motorolla format).

The third message is also Security Access, but the Sub-Function is 0x60 which is **sendKey**. The key value are the next two bytes and they are calculated by the service tool based on the securitySeed value. In other words, there is a challenge-response function that maps 0x118E to 0xC02C:
`f(0x118E) = 0xC02C`

From a cybersecurity stand point, this so-called security is insufficient to stop anything but an accidental session on the bus. 16-bits are far too few to be cryptographically sound. 

**THIS APPROACH IS NOT ACTUALLY SECURE!**

We could figure out $f$. How?

Finally, the last message is from the Engine with a positive response to the key exchange. Notice there are only 2 bytes in this response, 0x02. This means the bytes 11 8E 01 0E 03 have no meaning and you can see they were left over from the last transmission from the engine. They can be ignored. 

Now the security setup has been satisfied, and the session can continue.

##  Multiple Frame Message
The next block of messages make use of the ISO transport protocol. The first nibble in the messages are greater than zero. This is byte is called the protocol control information (PCI). 

In [18]:
# look at the next 3 messages
for i in range(10,13):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  10 0D 2E F1 5C 00 00 00
From:   0, To: 241,  30 08 14 11 8E 01 0E 03
From: 241, To:   0,  21 13 03 1C 00 00 00 00


The first message is from the OBD tool to the engine. The first nibble is a 1, which means it is the first frame in a multi frame message. The next 3 nibbles (12 bits) are the length of the message in bytes.

In [19]:
# Compute the message length
message_bytes = struct.unpack(">H",uds_messages[10]['data'][0:2])[0]
print(message_bytes)

4109


In [20]:
# Since we only want 3 nibbles, we have to mask off the initial 1:
message_bytes &= 0x0FFF
print(message_bytes)

13


In [21]:
0xD

13

So, we are expecting 13 bytes of data. The data count starts with the bytes immediatly following the count. There are 6 of the 13 bytes on the first line: `2E F1 5C 00 00 00`

The second line is from the engine to the tool and is a 0x30 Flow control PCI message. The value of 3 in the first nibble indicates it is a flow control message. The second nibble, 0, indicates it's clear to send. The options are 0 = Clear To Send, 1 = Wait, 2 = Overflow/abort. The next byte, 0x08, indicates the number of frames that may be sent before waiting for the next flow control frame. A value of zero allows the remaining frames to be sent without flow control or delay. The third byte, 0x14, is the Separation Time (ST) or the minimum time in milliseconds between frames. In this case, frames should be send 20 ms apart. 

In [22]:
0x14

20

In the last message, the remaining 7 of the 13 bytes are transmitted. The PCI nibble is 2 and the next nibble is 1, which means it is the first of the consecutive frames. The second nibble will increment as subsequent consecutive frames arrive. The last of the message is `13 03 1C 00 00 00 00`

In [23]:
full_message = bytes.fromhex('2E F1 5C 00 00 00 13 03 1C 00 00 00 00')
print('Message length:', len(full_message))
print(" ".join(["{:02X}".format(b) for b in full_message]))

Message length: 13
2E F1 5C 00 00 00 13 03 1C 00 00 00 00


This starts with a SID of 0x2E, which is **Write Data By Identififer**. The data identifier is 0xF15C and the data to write is 0x00 00 00 13 03 1C 00 00 00 00. The data identifier falls in the identificationOptionVehicleManufacturerSpecificDataIdentifier range.

In [24]:
#The next message
i=13
print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
      " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From:   0, To: 241,  03 6E F1 5C 8E 01 0E 03


This is a message from the engine that is 3 bytes long. It is a response to the Write Data By Identifier. Recall, 0x6E = 0x2E + 0x40. There is an acknowledgement of the identifier of 0xF15C. The  remaining bytes can be ignored.

## Routine Control

In [ ]:
# look at the next 2 messages exchanged
for i in range(14,16):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

This is from the OBD Tool sending 5 bytes. The SID is 0x31, which is for **Routine Control**. This typically means the tool is asking the ECU to do something. From 14229:

    The RoutineControl service is used by the client to execute a defined sequence of 
    steps and obtain any relevant results. There is a lot of flexibility with this service, 
    but typical usage may include functionality such as erasing memory, resetting or 
    learning adaptive data, running a self-test, overriding the normal server control 
    strategy, and controlling a server value to change over time including predefined 
    sequences (e.g., close convertible roof) to name a few. In general, when used to 
    control outputs this service is used for more complex type control whereas 
    inputOutputControlByIdentifier is used for relatively simple (e.g., static) output control.

The subsequent bytes inlude the Sub-Function, routineIdentifier, and routineControlOptionRecord.

There are three options for the Sub-Function:

| Value | Name  | Description | Mnemonic |
| --- | :-- | :-- | :-- |
| 0x01 | startRoutine | This parameter specifies that the server shall start the routine specified by the routineIdentifier. | STR |
| 0x02 | stopRoutine | This parameter specifies that the server shall stop the routine specified by the routineIdentifier. | STPR |
| 0x03 | requestRoutineResults | This parameter specifies that the server shall return result values of the routine specified by the routineIdentifier. | RRR

In this case, the Sub-Finction is 0x01, which means to start a routine. 
. 
The following bytes are 0x0304, which is the dataIdenfifier for the service routine. Appendix F of the ISO 14229-3 says this parameter falls within the following:

| Byte Value | Name  | Description | Mnemonic |
| --- | :-- | :-- | :-- |
| 0x0200 - 0xDFFF | vehicleManufacturerSpecific | This range of values is reserved for vehicle manufacturer specific use. | VMS_ |

We don't know what this is requesting, yet.

The last byte of the first message is a 0x01, which is the routineControlOptionRecord. We don't know this option means at this time, but the service tool does know.



In [25]:
# look at the next parameter exchanged
for i in range(16,22):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  03 22 F1 95 00 00 00 00
From:   0, To: 241,  10 16 62 F1 95 00 02 56
From: 241, To:   0,  30 08 00 00 00 00 00 00
From:   0, To: 241,  21 00 01 58 16 04 00 61
From:   0, To: 241,  22 0E 01 0E 0A 18 0C 00
From:   0, To: 241,  23 00 00 0E 0A 18 0C 00


After the tool has initiated the 0x0304 routineIdentifier, the OBD Tool asks the engine for the 0xF195 parameter using the Request Parameter by ID SID, 0x22. Appendix C says 0xF195 is:

*systemSupplierECUSoftwareVersionNumberDataIdentifier* This value shall be used to reference the system supplier specific ECU (server) software version number. Record data content and format shall be server specific and defined by the system supplier.

The response from the engine on the second line starts with an First Frame indicator 0x10 and the data length is 0x16 or 22 bytes. The first 6 bytes are `62 F1 95 00 02 56`. The 0x62 is the positive response to the data request by ID, 0x62 = 0x22 + 0x40.

A flow control message follows indicating to send upto the next 8 messages. 

Finally, the following three consecutive frames fill out the message. 

In [26]:
full_message = bytes.fromhex('62 F1 95 00 02 56 00 01 58 16 04 00 61 0E 01 0E 0A 18 0C 00 00 00')
print("Message length: ", len(full_message))

Message length:  22


In [27]:
0x16

22

### VIN

In [28]:
# look at the next 5 messages exchanged
for i in range(22,27):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  03 22 F1 A0 00 00 00 00
From:   0, To: 241,  10 14 62 F1 A0 31 46 55
From: 241, To:   0,  30 08 00 00 00 00 00 00
From:   0, To: 241,  21 42 47 44 44 52 38 43
From:   0, To: 241,  22 4C 42 46 33 30 30 30


This next set of messages starts with the OBD tool requesting another parameter by ID. The ID is 0xF1A0, which is

*identificationOptionVehicleManufacturerSpecific* - This range of values shall be used for vehicle manufacturer specific server/vehicle identification options.

The repsonse is 20 bytes long (0x14). The response to the SID is the following 3 bytes (0x62 0xF1 0xA0) and the data are `31 46 55 42 47 44 44 52 38 43 4C 42 46 33 30 30 30` 

In [29]:
bytes.fromhex("31 46 55 42 47 44 44 52 38 43 4C 42 46 33 30 30 30")

b'1FUBGDDR8CLBF3000'

This is clearly a VIN. This means we can tell that the VIN for the DDEC UDS service is the identifer in the 0xF1A0 position.

### Engine Serial Number

In [30]:
# look at the next 5 messages exchanged
for i in range(27,32):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  03 22 F1 80 00 00 00 00
From:   0, To: 241,  10 11 62 F1 80 34 37 32
From: 241, To:   0,  30 08 00 00 00 00 00 00
From:   0, To: 241,  21 39 30 33 53 30 30 38
From:   0, To: 241,  22 37 34 35 33 30 30 38


This exchange is the same as the previous VIN example, except the ID is 0xF180. The ISO 14229 Appendix C says this parameter is BootSoftwareIdentificationDataIdentifier. From the standard:
    
    This value shall be used to reference the vehicle manufacturer specific
    ECU boot software identification record. The first data byte of the record
    data shall be the numberOfModules that are reported. Following the
    numberOfModules the boot software identification(s) are reported. The
    format of the boot software identification structure shall be ECU specific
    and defined by the vehicle manufacturer.

The data are 17 bytes long. The first three of those bytes are used by the positive response SID (`62 F1 80`).

In [31]:
# Encode the BootSoftwarID
bytes.fromhex("34 37 32 39 30 33 53 30 30 38 37 34 35 33")

b'472903S0087453'

This is a case where the standard may not line up with the actual practice. This is becasue the data field that is displayed is actually the engine serial number. 
From the DDEC Reports PDF:
    
    Engine S/N: 472903S0087453

In [32]:
for i in range(32,39):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  03 22 F1 A1 00 00 00 00
From:   0, To: 241,  07 62 F1 A1 00 15 19 17
From: 241, To:   0,  03 22 F1 8C 00 00 00 00
From:   0, To: 241,  10 10 62 F1 8C 30 3B 2E
From: 241, To:   0,  30 08 00 00 00 00 00 00
From:   0, To: 241,  21 30 31 35 38 2E 30 30
From:   0, To: 241,  22 33 30 30 38 2E 30 30


In [33]:
for i in range(39,55):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  03 22 F1 16 00 00 00 00
From:   0, To: 241,  10 10 62 F1 16 30 30 32
From: 241, To:   0,  30 08 00 00 00 00 00 00
From:   0, To: 241,  21 34 34 36 38 32 30 32
From:   0, To: 241,  22 30 30 33 38 32 30 32
From: 241, To:   0,  03 22 F1 26 00 00 00 00
From:   0, To: 241,  10 10 62 F1 26 30 31 37
From: 241, To:   0,  30 08 00 00 00 00 00 00
From:   0, To: 241,  21 34 34 38 35 38 30 32
From:   0, To: 241,  22 30 30 31 35 38 30 32
From: 241, To:   0,  03 22 F1 00 00 00 00 00
From:   0, To: 241,  07 62 F1 00 02 01 0E 03
From: 241, To:   0,  03 22 F1 A2 00 00 00 00
From:   0, To: 241,  10 0B 62 F1 A2 21 2A 11
From: 241, To:   0,  30 08 00 00 00 00 00 00
From:   0, To: 241,  21 1C 04 03 13 00 2A 11


In [34]:
for i in range(55,62):
    print("From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

From: 241, To:   0,  03 22 F8 04 00 00 00 00
From:   0, To: 241,  10 13 62 F8 04 30 4D 34
From: 241, To:   0,  30 08 00 00 00 00 00 00
From:   0, To: 241,  21 30 58 30 30 30 30 30
From:   0, To: 241,  22 30 30 30 30 30 30 30
From: 241, To:   0,  03 22 F8 06 00 00 00 00
From:   0, To: 241,  07 62 F8 06 B7 A9 8A C9


After working through some of the log file request after request, we may find the following parameters based on their ID:

| Identifier | Bytes | ISO 14229 meaning | DDEC Reports Meaning |
| :-- | :-- | :-- | :-- |
| 0xF1A1 | `00 15 19 17` |  identificationOptionVehicleManufacturerSpecific | ?? |
| 0xF18C | `30 3B 2E 30 31 35 38 2E 30 30 33 30 30` | ECUSerialNumberDataIdentifier | Not Used |
| 0xF116 | `30 30 32 34 34 36 38 32 30 32 30 30 33` | identificationOptionVehicleManufacturerSpecific | Not Used |
| 0xF126 | `30 31 37 34 34 38 35 38 30 32 30 30 31` | identificationOptionVehicleManufacturerSpecific | Not Used |
| 0xF1A2 | `21 2A 11 1C 04 03 13 00` | identificationOptionVehicleManufacturerSpecific | ?? |
| 0xF804 | `30 4D 34 30 58 30 30 30 30 30 30 30 30 30 30 30` | OBDInfoTypeDataIdentifier | ?? | 
| 0xF806 | `B7 A9 8A C9` | OBDInfoTypeDataIdentifier | ?? | 

In [35]:
bytes.fromhex("30 3B 2E 30 31 35 38 2E 30 30 33 30 30")

b'0;.0158.00300'

In [36]:
bytes.fromhex("30 30 32 34 34 36 38 32 30 32 30 30 33")

b'0024468202003'

In [37]:
bytes.fromhex("30 31 37 34 34 38 35 38 30 32 30 30 31")

b'0174485802001'

In [38]:
bytes.fromhex("21 2A 11 1C 04 03 13 00")

b'!*\x11\x1c\x04\x03\x13\x00'

In [39]:
bytes.fromhex("30 4D 34 30 58 30 30 30 30 30 30 30 30 30 30 30")

b'0M40X00000000000'

## Negative Acknowledgment
There are many situations where the ECU will tell the Tool that it can't handle the request. Here's an example

In [40]:
#
for i in range(4851,5067):
    print(i, "From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

4851 From: 241, To:   0,  03 22 3E 01 00 00 00 00
4852 From:   0, To: 241,  03 7F 22 78 01 FF C8 C0
4853 From:   0, To: 241,  11 4F 62 3E 01 4A 01 05
4854 From: 241, To:   0,  30 08 00 00 00 00 00 00
4855 From:   0, To: 241,  21 FF 07 7D FF 7F 02 00
4856 From:   0, To: 241,  22 00 00 04 00 00 00 99
4857 From:   0, To: 241,  23 05 00 00 02 00 00 00
4858 From:   0, To: 241,  24 00 00 00 00 63 00 00
4859 From:   0, To: 241,  25 00 00 00 00 00 00 00
4860 From:   0, To: 241,  26 00 00 00 00 00 00 00
4861 From:   0, To: 241,  27 00 00 00 00 00 00 00
4862 From:   0, To: 241,  28 00 00 00 00 03 00 00
4863 From: 241, To:   0,  30 08 00 00 00 00 00 00
4864 From:   0, To: 241,  29 00 36 05 00 00 00 00
4865 From:   0, To: 241,  2A 00 00 00 00 00 00 00
4866 From:   0, To: 241,  2B 00 00 00 00 00 00 00
4867 From:   0, To: 241,  2C 00 00 00 00 00 00 00
4868 From:   0, To: 241,  2D 00 00 00 00 00 00 00
4869 From:   0, To: 241,  2E 00 00 1B 45 1C 14 00
4870 From:   0, To: 241,  2F 00 00 00 00 00 00 00


This block of the UDS Session starts with a Read Data By Identifier (0x22) request from the OBD-tool. The read request ID is 0x3E01. The ECU replies with a SID of 0x7F, which is the Negative Response SID. Following the neg. resp. is the service request identifier 0x22 and the negative response code (NRC), 0x78. According to Annex A.1 in ISO 14229, the NRC of 0x78 is:

   **requestCorrectlyReceived-ResponsePending**
    
    This NRC indicates that the request message was received correctly, and that all
    parameters in the request message were valid, but the action to be performed is
    not yet completed and the server is not yet ready to receive another request. As
    soon as the requested service has been completed, the server shall send a
    positive response message or negative response message with a response code
    different from this.
    The negative response message with this NRC may be repeated by the server
    until the requested service is completed and the final response message is sent.
    This NRC might impact the application layer timing parameter values. The detailed
    specification shall be included in the data link specific implementation document.
    This NRC shall only be used in a negative response message if the server will not
    be able to receive further request messages from the client while completing the
    requested diagnostic service.
    When this NRC is used, the server shall always send a final response (positive or
    negative) independent of the suppressPosRspMsgIndicationBit value or the
    suppress requirement for responses with NRCs SNS, SFNS, SNSIAS, SFNSIAS
    and ROOR on functionally addressed requests.
    A typical example where this NRC may be used is when the client has sent a
    request message, which includes data to be programmed or erased in flash
    memory of the server. If the programming/erasing routine (usually executed out of
    RAM) is not able to support serial communication while writing to the flash memory
    the server shall send a negative response message with this response code.
    This NRC is in general supported by each diagnostic service, as not otherwise
    stated in the data link specific implementation document, therefore it is not listed in
    the list of applicable response codes of the diagnostic services.

After the NRC of 0x78 was sent, the data starts flowing from the engine controller. In essence, the NRC of 0x78 tells the service tool to wait for a bit while I gather the data.

Let's look at the timing of the requests:

In [41]:
for i in range(4851,4858):
    print(i, uds_messages[i]['timestamp'],"From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da']),
          " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']]))

4851 1531228571.429646 From: 241, To:   0,  03 22 3E 01 00 00 00 00
4852 1531228571.453436 From:   0, To: 241,  03 7F 22 78 01 FF C8 C0
4853 1531228572.223366 From:   0, To: 241,  11 4F 62 3E 01 4A 01 05
4854 1531228572.224596 From: 241, To:   0,  30 08 00 00 00 00 00 00
4855 1531228572.225715 From:   0, To: 241,  21 FF 07 7D FF 7F 02 00
4856 1531228572.226846 From:   0, To: 241,  22 00 00 04 00 00 00 99
4857 1531228572.227446 From:   0, To: 241,  23 05 00 00 02 00 00 00


In [42]:
# Delta time.
print('index', 'elapsed,','  delta', ' Source,      Dest', '   Data Bytes')
start_time = uds_messages[4851]['timestamp']
last_time = start_time
for i in range(4851,4858):
    ts = uds_messages[i]['timestamp']
    elapsesd_time = ts-start_time
    delta_t = ts - last_time
    last_time = ts
    addr = "From: {:3d}, To: {:3d}, ".format(uds_messages[i]['sa'],uds_messages[i]['da'])
    frame = " ".join(["{:02X}".format(b) for b in uds_messages[i]['data']])
        
    print(i, "{:0.6f}, {:0.6f}".format(elapsesd_time, delta_t), addr, frame)

index elapsed,   delta  Source,      Dest    Data Bytes
4851 0.000000, 0.000000 From: 241, To:   0,  03 22 3E 01 00 00 00 00
4852 0.023790, 0.023790 From:   0, To: 241,  03 7F 22 78 01 FF C8 C0
4853 0.793720, 0.769930 From:   0, To: 241,  11 4F 62 3E 01 4A 01 05
4854 0.794950, 0.001230 From: 241, To:   0,  30 08 00 00 00 00 00 00
4855 0.796069, 0.001119 From:   0, To: 241,  21 FF 07 7D FF 7F 02 00
4856 0.797200, 0.001131 From:   0, To: 241,  22 00 00 04 00 00 00 99
4857 0.797800, 0.000600 From:   0, To: 241,  23 05 00 00 02 00 00 00


Notice the NRC of 0x78 came just passed 20 milliseconds. Recall the session response (SID = 0x50) from the ECU said the timing was 20 milliseconds. The ECU takes almost 800 milliseconds to start a response to the request. It's a large response taking up 335 (0x014F) bytes.

In [43]:
0x14f

335

## Building a parser for UDS traffic
Performing the analysis by hand, like we've done so far is challenging. Instead, we need to be able to create a routine that can automatically build long messages for UDS transport. There are 2 ways of doing this: 1) build a system in Python that can handle UDS traffic, or 2) use the built-in SocketCAN tools that can handle ISO-TP.

Let's assume we are running this notebook on the BeagleBone or Linux machine. If you are on Windows, you can't take advantage of the virtual CAN channels.

1. Open a new terminal session (PuTTy or SSH)

`ssh -o ServerAliveInterval=60 student@cybertruck1.engr.colostate.edu`

2. Clone the repository

`git clone https://github.com/SystemsCyber/TruckCapeProjects`

3. Change directories

`cd TruckCapeProjects/Jupyter/`

4. Download the data

`wget https://www.engr.colostate.edu/~jdaily/J1939/files/candump_DDECReportsExtaction032819.txt`

5. Check to be sure vcan5 is available.

`ifconfig | grep vcan5`

6. If not, startup a new vcan5 interface. 

`modprobe vcan`

`sudo ip link add dev vcan5 type vcan`

`sudo ip link set up vcan5`

7. Try to look at the data on vcan5: 

`candump vcan5`

8. If there is no traffic from the above, start the CAN replay service using canplayer. Only run this command if `candump vcan5` shows nothing. 

`canplayer -l i -I candump_DDECReportsExtaction032819.txt vcan5=can1 &`

9. We can use the can utils to get these data at the command line:

`isotpsniffer -s 18DAF100 -d 18DA00F1 vcan5`

Let's try to get the same type of data from the sockets.

Reference: https://can-isotp.readthedocs.io/en/latest/isotp/socket.html

In [ ]:
#Minimal Use the ISOTP connection to see data
import socket
import struct
s1 = socket.socket(socket.AF_CAN, socket.SOCK_DGRAM, socket.CAN_ISOTP)

s1.bind(("vcan5", 0x98DAF100, 0x98DA00F1)) #rxid, txid. This is to get up and running

In [ ]:
for i in range(20):
    print(s1.recv(4095))

In [ ]:
# We can also use the raw socket interface too.
s2 = socket.socket(socket.AF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
s2.bind(("vcan5",))

In [ ]:
for i in range(20):
    print(s2.recv(16))

In [ ]:
# Let's broadcast on an empty channel to see how SocketCAN works with ISOTP
SOL_CAN_ISOTP = 106 # These constants exist in the module header, not in Python.
CAN_ISOTP_RECV_FC = 2

s3 = socket.socket(socket.AF_CAN, socket.SOCK_DGRAM, socket.CAN_ISOTP)

s3.setsockopt(SOL_CAN_ISOTP, CAN_ISOTP_RECV_FC, struct.pack("=BBB", 0x8, 3, 20)) #bs, stmin, wftmax
s3.bind(("vcan1",0x98DAF903, 0x98DA03F9)) # Use a transmission (3) and a diagnostic connector (0xF9)
s3.send(b"12345678 Here is a long message")

```
debian@beaglebone:~/TruckCapeProjects/Jupyter$ candump vcan0
  vcan0  18DA00F1   [8]  10 16 48 65 72 65 20 69
  vcan0  18DA00F1   [8]  10 16 48 65 72 65 20 69

```
This only sends the first frame out. Why didn't it work? 

There needs to be another listener to respond. 

In [ ]:
s4 = socket.socket(socket.AF_CAN, socket.SOCK_DGRAM, socket.CAN_ISOTP)
s4.setsockopt(SOL_CAN_ISOTP, CAN_ISOTP_RECV_FC, struct.pack("=BBB", 0x8, 3, 20)) #bs, stmin, wftmax
s4.bind(("vcan1",0x98DA03F9, 0x98DAF903)) #Switch sender and receiver. These must match.
s3.send(b"12345678 Here is a long message")

Monitor using candump:

```
debian@beaglebone:~/TruckCapeProjects/Jupyter$ candump -a vcan0
  vcan0  18DA03F9   [8]  10 1F 31 32 33 34 35 36   '..123456'
  vcan0  18DAF903   [3]  30 08 03                  '0..'
  vcan0  18DA03F9   [8]  21 37 38 20 48 65 72 65   '!78 Here'
  vcan0  18DA03F9   [8]  22 20 69 73 20 61 20 6C   '" is a l'
  vcan0  18DA03F9   [8]  23 6F 6E 67 20 6D 65 73   '#ong mes'
  vcan0  18DA03F9   [5]  24 73 61 67 65            '$sage'

```

Both sockets are needed to work with the data.

### Send a photo
Let's send this photo over the CAN bus:
![CSURam.jpg](CSURam.jpg)

In [44]:
#Open an image file
with open("CSURam.jpg",'rb') as img_file:
    image = img_file.read()
#Show the number of bytes
print(len(image))

FileNotFoundError: [Errno 2] No such file or directory: 'CSURam.jpg'

In [ ]:
s3.send(image)

The network looks like this:
```candump -a vcan0
  vcan0  18DA03F9   [8]  1B F9 FF D8 FF E0 00 10   '........'
  vcan0  18DAF903   [3]  30 08 03                  '0..'
  vcan0  18DA03F9   [8]  21 4A 46 49 46 00 01 01   '!JFIF...'
  vcan0  18DA03F9   [8]  22 02 00 EC 00 EC 00 00   '".......'
  vcan0  18DA03F9   [8]  23 FF DB 00 43 00 03 02   '#...C...'
  vcan0  18DA03F9   [8]  24 02 02 02 02 03 02 02   '$.......'
  vcan0  18DA03F9   [8]  25 02 03 03 03 03 04 06   '%.......'
  vcan0  18DA03F9   [8]  26 04 04 04 04 04 08 06   '&.......'
  vcan0  18DA03F9   [8]  27 06 05 06 09 08 0A 0A   ''.......'
  vcan0  18DA03F9   [8]  28 09 08 09 09 0A 0C 0F   '(.......'
  vcan0  18DAF903   [3]  30 08 03                  '0..'
  vcan0  18DA03F9   [8]  29 0C 0A 0B 0E 0B 09 09   ').......'
  vcan0  18DA03F9   [8]  2A 0D 11 0D 0E 0F 10 10   '*.......'
  vcan0  18DA03F9   [8]  2B 11 10 0A 0C 12 13 12   '+.......'
  ...
```
Notice the number of messages that are sent before the control flow is 8. This is tunable.

In [ ]:
s4 = socket.socket(socket.AF_CAN, socket.SOCK_DGRAM, socket.CAN_ISOTP)
#Change the block size
s4.setsockopt(SOL_CAN_ISOTP, CAN_ISOTP_RECV_FC, struct.pack("=BBB", 0x20, 3, 20)) #bs, stmin, wftmax
s4.bind(("vcan1",0x98DA03F9, 0x98DAF903))

In [ ]:
#Run this command again while running candump vcan0
s3.send(image)

The output from candump show the receiver waits for blocks of 32
```
debian@beaglebone:~/TruckCapeProjects/Jupyter$ candump -a vcan0
  vcan0  18DA03F9   [8]  1B F9 FF D8 FF E0 00 10   '........'
  vcan0  18DAF903   [3]  30 20 03                  '0 .'
  vcan0  18DA03F9   [8]  21 4A 46 49 46 00 01 01   '!JFIF...'
  vcan0  18DA03F9   [8]  22 02 00 EC 00 EC 00 00   '".......'
  vcan0  18DA03F9   [8]  23 FF DB 00 43 00 03 02   '#...C...'
  vcan0  18DA03F9   [8]  24 02 02 02 02 03 02 02   '$.......'
  vcan0  18DA03F9   [8]  25 02 03 03 03 03 04 06   '%.......'
  vcan0  18DA03F9   [8]  26 04 04 04 04 04 08 06   '&.......'
  vcan0  18DA03F9   [8]  27 06 05 06 09 08 0A 0A   ''.......'
  vcan0  18DA03F9   [8]  28 09 08 09 09 0A 0C 0F   '(.......'
  vcan0  18DA03F9   [8]  29 0C 0A 0B 0E 0B 09 09   ').......'
  vcan0  18DA03F9   [8]  2A 0D 11 0D 0E 0F 10 10   '*.......'
  vcan0  18DA03F9   [8]  2B 11 10 0A 0C 12 13 12   '+.......'
  vcan0  18DA03F9   [8]  2C 10 13 0F 10 10 10 FF   ',.......'
  vcan0  18DA03F9   [8]  2D DB 00 43 01 03 03 03   '-..C....'
  vcan0  18DA03F9   [8]  2E 04 03 04 08 04 04 08   '........'
  vcan0  18DA03F9   [8]  2F 10 0B 09 0B 10 10 10   '/.......'
  vcan0  18DA03F9   [8]  20 10 10 10 10 10 10 10   ' .......'
  vcan0  18DA03F9   [8]  21 10 10 10 10 10 10 10   '!.......'
  vcan0  18DA03F9   [8]  22 10 10 10 10 10 10 10   '".......'
  vcan0  18DA03F9   [8]  23 10 10 10 10 10 10 10   '#.......'
  vcan0  18DA03F9   [8]  24 10 10 10 10 10 10 10   '$.......'
  vcan0  18DA03F9   [8]  25 10 10 10 10 10 10 10   '%.......'
  vcan0  18DA03F9   [8]  26 10 10 10 10 10 FF C0   '&.......'
  vcan0  18DA03F9   [8]  27 00 11 08 00 32 00 32   ''....2.2'
  vcan0  18DA03F9   [8]  28 03 01 11 00 02 11 01   '(.......'
  vcan0  18DA03F9   [8]  29 03 11 01 FF C4 00 1B   ').......'
  vcan0  18DA03F9   [8]  2A 00 00 02 02 03 01 00   '*.......'
  vcan0  18DA03F9   [8]  2B 00 00 00 00 00 00 00   '+.......'
  vcan0  18DA03F9   [8]  2C 00 00 00 07 08 06 09   ',.......'
  vcan0  18DA03F9   [8]  2D 00 03 05 04 FF C4 00   '-.......'
  vcan0  18DA03F9   [8]  2E 32 10 00 01 04 01 03   '.2......'
  vcan0  18DA03F9   [8]  2F 04 01 02 06 01 03 05   '/.......'
  vcan0  18DA03F9   [8]  20 00 00 00 00 02 01 03   ' .......'
  vcan0  18DAF903   [3]  30 20 03                  '0 .'
  vcan0  18DA03F9   [8]  21 04 05 06 07 11 12 00   '!.......'
```
In this case, the counter on the second nibble of the consecutive frame message cycles through 16 counts twice. 

In [ ]:
#Send the message to the socket and vcan.
s3.send(image)

#Receive the message
new_image = s4.recv(4095)
print(len(new_image))

In [ ]:
with open('newRam.jpg','wb') as new_file:
    new_file.write(new_image)

If the new image did not come through correctly, then the following image will not show up:
![Bad Image](newRam.jpg)

### Summary
Using an example of a DDEC Reports, we were introduced to the UDS communication system. This whowe different service identitfier, data ids, and negative response codes. We also used the SocketCAN ISO TP feature to send messages across the bus.